In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# Check if the key is the same as the one you think you updated
api_key = os.getenv('GROQ_API_KEY')
print(f"Current key (last 10 chars): ...{api_key[-10:]}")
print(f"Key starts with 'gsk_': {api_key.startswith('gsk_') if api_key else 'No key found'}")

Current key (last 10 chars): ...Ci0JLQJj6O
Key starts with 'gsk_': True


In [2]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

# Initialize the model
llm = ChatGroq(model="llama-3.1-8b-instant")

# Get a response
response = llm.invoke("Hello! How are you today?")
print(response.content)

I'm just a language model, so I don't have emotions or feelings like humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?


In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
urls=[
    "https://langchain-ai.github.io/langgraph/tutorials/introduction/",
    "https://langchain-ai.github.io/langgraph/tutorials/workflows/",
    "https://langchain-ai.github.io/langgraph/how-tos/map-reduce/"
]

docs=[WebBaseLoader(url).load() for url in urls ]
docs

In [ ]:
doc_list=[item for sublist in docs for item in sublist]
print(doc_list)
text_splitter= RecursiveCharacterTextSplitter(chunk_size=900,chunk_overlap=100)
doc_splitter=text_splitter.split_documents(doc_list)
print(doc_splitter)

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
Embeddings_ollama=(OllamaEmbeddings(model="mxbai-embed-large:335m"))
Embeddings_ollama

vectorstorage=FAISS.from_documents(
    documents=doc_splitter,
    embedding=Embeddings_ollama
)
retriver=vectorstorage.as_retriever()

In [ ]:
retriver.invoke("what is langraph")

In [8]:
##
from langchain.tools.retriever import create_retriever_tool

retriver_tool=create_retriever_tool(
    retriver,
    "retriver_vector_db_bolg",
    "Search anf run information about Langgraph"

)

In [9]:
retriver_tool

Tool(name='retriver_vector_db_bolg', description='Search anf run information about Langgraph', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000027C0107CD60>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000027C01056270>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000027C0107E160>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000027C01056270>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), d

In [ ]:
langchain_urls = [
    "https://python.langchain.com/docs/tutorials/",
    "https://python.langchain.com/docs/tutorials/chatbot/",
    "https://python.langchain.com/docs/tutorials/qa_chat_history/",
]
docs=[WebBaseLoader(url).load() for url in langchain_urls ]
docs

In [ ]:

from langchain_community.embeddings import OllamaEmbeddings

doc_listing=[item for sublist in docs for item in sublist]
print(doc_listing)

text_split= RecursiveCharacterTextSplitter(
    chunk_size=900,chunk_overlap=100
)

doc_split=text_split.split_documents(doc_listing)
print(doc_split)

vectorstoragelangchain=FAISS.from_documents(
    documents=doc_split,
    embedding=Embeddings_ollama
)
retriverlangchain=vectorstoragelangchain.as_retriever()


In [12]:
retriver_tool_cahin=create_retriever_tool(
    retriver,
    "retriver_vector_langchin_bolg",
    "Search and run information about LangChain"

)

In [13]:
tools=[retriver_tool,retriver_tool_cahin]


In [14]:
from typing import Annotated,Sequence,Literal
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage,HumanMessage,AIMessage
from langgraph.graph.message import add_messages
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel,Field
from langchain import hub

class AgentState(TypedDict):
    messages:Annotated[Sequence[BaseMessage],add_messages]

In [15]:
def agent(state):
    """
    Invokes the agent model to generate a response based on the current state. Given
    the question, it will decide to retrieve using the retriever tool, or simply end.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with the agent response appended to messages
    """
    print("---CALL AGENT---")
    messages = state["messages"]
    model = ChatGroq(model="llama-3.1-8b-instant")
    model = model.bind_tools(tools)
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [16]:
### Edges
def grade_documents(state) -> Literal["generate", "rewrite"]:
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (messages): The current state

    Returns:
        str: A decision for whether the documents are relevant or not
    """

    print("---CHECK RELEVANCE---")

    # Data model
    class grade(BaseModel):
        """Binary score for relevance check."""

        binary_score: str = Field(description="Relevance score 'yes' or 'no'")

    # LLM
    model = ChatGroq(model="llama-3.1-8b-instant")

    # LLM with tool and validation
    llm_with_tool = model.with_structured_output(grade)

    # Prompt
    prompt = PromptTemplate(
        template="""You are a grader assessing relevance of a retrieved document to a user question. \n 
        Here is the retrieved document: \n\n {context} \n\n
        Here is the user question: {question} \n
        If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
        Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.""",
        input_variables=["context", "question"],
    )

    # Chain
    chain = prompt | llm_with_tool

    messages = state["messages"]
    last_message = messages[-1]

    question = messages[0].content
    docs = last_message.content

    scored_result = chain.invoke({"question": question, "context": docs})

    score = scored_result.binary_score

    if score == "yes":
        print("---DECISION: DOCS RELEVANT---")
        return "generate"

    else:
        print("---DECISION: DOCS NOT RELEVANT---")
        print(score)
        return "rewrite"

In [17]:
def generate(state) :
    """
    Generate the answer

    Args:
        state (messages): The current state

    Returns:
        dict:The updated message
    """

    print("--Generate--")
    messages = state["messages"]
    question = messages[0].content
    last_message = messages[-1]

    docs=last_message.content

    prompt=hub.pull("rlm/rag-prompt")

    llm=ChatGroq(model="llama-3.1-8b-instant")

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)
    
    rag_chain=prompt |llm | StrOutputParser()

    response=rag_chain.invoke({"context":docs,"question":question})
    #return {"messages":[response]}
    return {"messages":[AIMessage(content=response)]}

In [18]:
def rewrite(state):
    """
    Transform the query to produce a better question.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with re-phrased question
    """

    print("---TRANSFORM QUERY---")
    messages = state["messages"]
    question = messages[0].content

    msg = [
        HumanMessage(
            content=f""" \n 
    Look at the input and try to reason about the underlying semantic intent / meaning. \n 
    Here is the initial question:
    \n ------- \n
    {question} 
    \n ------- \n
    Formulate an improved question: """,
        )
    ]

    # Grader
    model = ChatGroq(model="llama-3.1-8b-instant")
    response = model.invoke(msg)
    return {"messages": [response]}

In [26]:
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

# Define a new graph
workflow = StateGraph(AgentState)

# Define the nodes we will cycle between
workflow.add_node("agent", agent)  # agent
retrieve = ToolNode([retriver_tool,retriver_tool_cahin])
workflow.add_node("retrieve", retrieve)  # retrieval
workflow.add_node("rewrite", rewrite)  # Re-writing the question
workflow.add_node(
    "generate", generate
)  # Generating a response after we know the documents are relevant
# Call agent node to decide to retrieve or not
workflow.add_edge(START, "agent")

# Decide whether to retrieve
workflow.add_conditional_edges(
    "agent",
    # Assess agent decision
    tools_condition,
    {
        # Translate the condition outputs to nodes in our graph
        "tools": "retrieve",
        END: END,
    },
)

# Edges taken after the `action` node is called.
# workflow.add_conditional_edges(
#     "retrieve",
#     # Assess agent decision
#     grade_documents,
# )

workflow.add_conditional_edges(
    "retrieve",
    grade_documents,
    {
        "generate": "generate",
        "rewrite": "rewrite",
    }
)

workflow.add_edge("generate", END)
workflow.add_edge("rewrite", "agent")

# Compile
graph = workflow.compile()
from IPython.display import Image, display
#display(Image(graph.get_graph(xray=True).draw_mermaid_png()))


In [ ]:

from langchain_core.messages import HumanMessage

# The input must be a list of BaseMessage objects
graph.invoke({"messages": [HumanMessage(content="What is Langraph?")]})

In [ ]:
graph.invoke({"messages": [HumanMessage(content="What is LangChain?")]})

In [ ]:
graph.invoke({"messages": [HumanMessage(content="What is Machine Learning?")]})